In [25]:
"""
Load a BehaviourStore setup json and interactively edit it.
"""
#fom BluenetTestSuite import ...
from ipywidgets import Layout, VBox, HBox, Accordion, Tab
from ipywidgets import IntRangeSlider, FloatText, IntSlider, BoundedIntText
from ipywidgets import Button, ToggleButton, ToggleButtons, Checkbox
from ipywidgets import jslink

In [26]:
def MakeHBox(left, left_width, middle, middle_width, right, right_width):
    hbox_layout = Layout(
        overflow='scroll',
        height='',
        margin='15px 15px 0 0',
        flex_flow='row',
        display='stretch')
    
    marge = '5px'
    columns = [
        VBox(children = left, layout = Layout(width = left_width, margin=marge)),
        VBox(children = middle, layout = Layout(width = middle_width, margin=marge)),
        VBox(children = right, layout = Layout(width = right_width, margin=marge))
    ]
    
    return HBox(children = columns, layout = hbox_layout)


def MetaDataSummary():
    indexfield = BoundedIntText(
        value=0,
        min=0,
        max=49,
        step=1,
        tooltip="index in behaviourstore",
        disabled=False,
        layout=Layout(width='100%')
    )
    
    return [indexfield]

def MetaDataDetails():
    return []

def BehaviourOverviewSummary():
    """
    Returns list of widgets for single line description of a behaviour
    """
    fromuntilfield = IntRangeSlider(
        value=[9*60*60, 18*60*60],
        min=0,
        max=24*60*60,
        step=60,
        disabled=False,
        continuous_update=False,
        orientation='horizontal',
        readout=False,
        readout_format='.1f',
        layout=Layout(width='100%')
    )
    
    fromfield = BoundedIntText(
        value=9*60*60,
        min=0,
        max=24*60*60,
        description='From:',
        disabled=False,
    )
    untilfield = BoundedIntText(
        value=18*60*60,
        min=0,
        max=24*60*60,
        description='Until:',
        disabled=False,
    )
    
    def on_range_field_change(change):
        fromval,untilval = change['new']
        fromfield.value = fromval
        untilfield.value = untilval
        
    def on_from_field_change(change):
        prev_from_until = fromuntilfield.value
        next_from_until = (change['new'], prev_from_until[1])
        
        if next_from_until[0] <= next_from_until[1]:
            fromuntilfield.value = next_from_until
    
    def on_until_field_change(change):
        prev_from_until = fromuntilfield.value
        next_from_until = (prev_from_until[0], change['new'])
        
        if next_from_until[0] <= next_from_until[1]:
            fromuntilfield.value = next_from_until
        
    fromuntilfield.observe(on_range_field_change, names='value')
    fromfield.observe(on_from_field_change, names='value')
    untilfield.observe(on_until_field_change, names='value')
    
    def update_summary(behaviour_settings_dict):
        pass
    
    return [fromuntilfield, fromfield, untilfield], update_summary

def BehaviourOverviewDetails():
    """
    Returns list of widgets for multi line description of a behaviour
    """    
    intensityfield = IntSlider(
        value=100,
        min=0,
        max=100,
        description='Intensity:',
        disabled=False,
        orientation='horizontal',
    )
    
    # only necessary because the range slider doesn't include reversed ranges...
    fromuntil_reversed_field = Checkbox(
        value=False,
        description='Midnight:',
        tooltip='Reverses from/until times when checked',
        disabled=False,
        indent=False
    )
    
    typefield = ToggleButtons(
        options=['Switch', 'ExtendedSwitch', 'Twilight'],
        description='Type:',
        disabled=False,
        button_style='info',
        layout=Layout(width='100%')
    )
    
    def get_behaviour_settings_dict():
            settings = dict()
            settings['intensity'] = intensityfield.value
            settings['type'] = typefield.value
            settings['fromuntil_reversed'] = fromuntil_reveresed_field.value
            
    
    return [intensityfield, fromuntil_reversed_field, typefield], get_behaviour_settings_dict


toolbar_icon_expand = 'bars'
toolbar_icon_ellipsis = 'ellipsis-h'
toolbar_icon_pencil = 'pencil'
toolbar_icon_cog = 'cog'

toolbar_icon_edit = toolbar_icon_cog
toolbar_icon_collapse = 'minus' #'window-minimize'
toolbar_icon_save = 'fa-floppy-o'
toolbar_icon_delete = 'fa-trash-o'
toolbar_icon_reload = 'fa-undo'

def ToolbarSummary():
    editbutton = ToggleButton(
        value = False,  # meaning 'collapsed'
        tooltip='Expand',
        disabled=False,
        icon=toolbar_icon_edit,
        layout=Layout(width='100%')
    )
    
    return [editbutton]
    
def ToolbarDetails():
    deletebutton = Button(
        tooltip='Delete',
        disabled=False,
        icon=toolbar_icon_delete,
        layout=Layout(width='100%')
    )
    
    savebutton = Button(
        tooltip='Save',
        disabled=False,
        icon=toolbar_icon_save,
        layout=Layout(width='100%')
    )
    
    undobutton = Button(
        tooltip='Reload',
        disabled=False,
        icon=toolbar_icon_reload,
        layout=Layout(width='100%')
    )
    
    return [savebutton, undobutton, deletebutton]

            
def BehaviourEntryEditor():
    """
    Returns HBox for Behaviour with additional meta-operations
    """
    summary_meta = MetaDataSummary()
    summary_overview, overview_update_callback = BehaviourOverviewSummary()
    summary_toolbar = ToolbarSummary()
    
    details_meta = MetaDataDetails()
    details_overview, overview_get_beh = BehaviourOverviewDetails()
    details_toolbar = ToolbarDetails()
    
    summary = MakeHBox(summary_meta, '5%', summary_overview, '90%', summary_toolbar, '5%')
    details = MakeHBox(details_meta, '5%', details_overview, '90%', details_toolbar, '5%')

    entry_editor = VBox([summary])
    
    editbutton = summary_toolbar[0]
    
    def toggle_detail_widgets(b):
        if editbutton.value:
            entry_editor.children = [summary, details]
            editbutton.tooltip = "Collapse"     
            editbutton.icon = toolbar_icon_collapse
        else:
            entry_editor.children = [summary]
            editbutton.tooltip = "Expand"
            editbutton.icon = toolbar_icon_edit

    editbutton.observe(toggle_detail_widgets, 'value')
    
    
    
    return entry_editor

In [27]:
VBox([BehaviourEntryEditor() for i in range(3)])

ValueError: too many values to unpack (expected 2)